### Julia implemention of all v.s. all BGEE expression profile matching 

The python ```BGEE_coexpression_generate-final``` notebook 
already produces the input and consumes the output
so this notenook is pretty minimal.

In [2]:
import YAML

INFO: Recompiling stale cache file /home/tomc/.julia/lib/v0.5/YAML.ji for module YAML.


In [3]:
# sed 's|!!python/object/apply:array.array||g;s|- b||g' gene_tissue_present.yaml | tr '-' ' ' > gene_tissue_present_stripped.yaml
#
data = YAML.load(open("gene_tissue_present_stripped.yaml"));
# damm that yaml load is slow

In [52]:
# get the input properly typed
gene_profile = convert(Dict{String,BitArray}, data);

In [56]:
# not strictly needed but split off the keys for look
genes = collect(keys(gene_profile));

In [1]:
genlen = length(genes)

LoadError: UndefVarError: genes not defined

In [ ]:
genes[73]

In [58]:
gene_profile[genes[73]]

251

In [ ]:
sum(gene_profile[genes[73]])

In [59]:
sum(gene_profile[genes[73]] & gene_profile[genes[730]])

221

In [231]:
# datastructure for results
pairwise_common = Dict{Pair{String, String}, UInt64}()

Dict{Pair{String,String},UInt64} with 0 entries

In [233]:
# allow one mismatch per (half of ffactor) tissues
ffactor = 80 # magic to allow one mismatch per "not too often"

for a in 1:genlen-1
    a_name = genes[a]
    a_len = sum(gene_profile[a_name])
    if 0 == a % 1000
        println(a, "\t", now(), " yeilds ", length(pairwise_common))  
        end    
    for b in a+1:genlen
        together = sum(gene_profile[a_name] & gene_profile[genes[b]])<<1
        fugde = floor(together/ffactor)
        separate = a_len + sum(gene_profile[genes[b]])
        if together > 0 && fugde + together >= separate
            pairwise_common[a_name => genes[b]] = together>>1
        end
    end                
end
println(genlen,"\t", now(), " yeilds ", length(pairwise_common))

1000	2017-10-11T15:47:18.484 yeilds 123296
2000	2017-10-11T15:49:12.555 yeilds 249514
3000	2017-10-11T15:51:03.843 yeilds 379383
4000	2017-10-11T15:52:54.61 yeilds 474093
5000	2017-10-11T15:54:42.727 yeilds 586516
6000	2017-10-11T15:56:28.341 yeilds 693016
7000	2017-10-11T15:58:13.447 yeilds 804918
8000	2017-10-11T15:59:54.489 yeilds 895589
9000	2017-10-11T16:01:33.862 yeilds 991449
10000	2017-10-11T16:03:12.003 yeilds 1083231
11000	2017-10-11T16:04:50.773 yeilds 1167823
12000	2017-10-11T16:06:25.92 yeilds 1280135
13000	2017-10-11T16:07:57.389 yeilds 1373576
14000	2017-10-11T16:09:29.727 yeilds 1463649
15000	2017-10-11T16:10:58.294 yeilds 1539351
16000	2017-10-11T16:12:24.776 yeilds 1608510
17000	2017-10-11T16:13:49.854 yeilds 1683270
18000	2017-10-11T16:15:12.378 yeilds 1779493
19000	2017-10-11T16:16:35.216 yeilds 1864808
20000	2017-10-11T16:17:53.85 yeilds 1940030
21000	2017-10-11T16:19:11.056 yeilds 2013726
22000	2017-10-11T16:20:28.516 yeilds 2087800
23000	2017-10-11T16:21:40.476 y

In [234]:
length(pairwise_common)

3450434

In [235]:
# average edges per node
length(pairwise_common)/genlen

58.31785146874894

In [236]:
fh = open("gene_gene_tissues.tab","w")
for key in keys(pairwise_common)
    println(fh, key.first,"\t", key.second, "\t", pairwise_common[key])
end   
close(fh)
# blindingly fast compared to processing yaml